# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data_df = pd.read_csv('../WeatherPy/output_data/city_data.csv')
weather_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 547 entries, 0 to 546
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City        547 non-null    object 
 1   Lat         547 non-null    float64
 2   Lng         547 non-null    float64
 3   Max Temp    547 non-null    float64
 4   Humidity    547 non-null    int64  
 5   Cloudiness  547 non-null    int64  
 6   Wind Speed  547 non-null    float64
 7   Country     544 non-null    object 
 8   Date        547 non-null    int64  
dtypes: float64(4), int64(3), object(2)
memory usage: 38.6+ KB


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#configuring the gmap
gmaps.configure(api_key=g_key)

In [4]:
#store lat and lng as floats
locations = weather_data_df[["Lat", "Lng"]].astype(float)

#store humidity as float
humidity = weather_data_df['Humidity'].astype(float)

In [5]:
#creating the heatmap
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [13]:
perf_conditions_df =  weather_data_df.loc[(weather_data_df['Max Temp'] < 80) & 
                                          (weather_data_df['Max Temp'] > 70) & 
                                          (weather_data_df['Wind Speed'] < 10) &  
                                          (weather_data_df['Cloudiness'] == 0)]
perf_conditions_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
219,Poum,-20.23,164.02,74.62,69,0,6.44,NC,1603211318
225,Ibrā’,22.69,58.53,70.03,32,0,1.57,OM,1603211766
248,Mutare,-18.97,32.67,73.89,29,0,4.05,ZW,1603211771
281,Umri,26.51,78.94,76.59,19,0,2.55,IN,1603211778
327,Sur,22.57,59.53,75.92,61,0,4.85,OM,1603211477


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [20]:
hotel_df = perf_conditions_df[['City','Country', 'Lat', 'Lng']]
hotel_df['Hotel'] = ' '
hotel_df.head()

<ipython-input-20-9704b2efa447>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel'] = ' '


,City,Country,Lat,Lng,Hotel
219,Poum,NC,-20.23,164.02,
225,Ibrā’,OM,22.69,58.53,
248,Mutare,ZW,-18.97,32.67,
281,Umri,IN,26.51,78.94,
327,Sur,OM,22.57,59.53,


In [ ]:
# Set parameters to search for a hotel
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Iterate through 
for index, row in hotel_df.iterrows():
# get lat, lng from df

    params["location"] =    # lat,lon from above
# Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# make request and print url
    name_address = requests.get(base_url, params=params)
    
# convert to json


   # Grab the first hotel from the results and store the name
    try:
        
    except:


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
